In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Resizing, Input, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot
import numpy as np
from sklearn.utils import shuffle

In [ ]:
classes = ['drink water','eat meal','brush teeth','brush hair','drop','pick up','throw','sit down',
           'stand up','clapping','reading','writing','tear up paper','put on jacket','take off jacket',
           'put on a shoe','take off a shoe','put on glasses','take off glasses','put on a hat cap','take off a hat cap',
           'cheer up','hand waving','kicking something','reach into pocket','hopping','jump up','phone call',
           'play with phone  tablet','type on a keyboard','point to something','taking a selfie','check time',
           'rub two hands','nod head bow','shake head','wipe face','salute','put palms together','cross hands in front',
           'sneeze cough','staggering','falling down','headache','chest pain','back pain','neck pain','nausea vomiting',
           'fan self','punch slap','kicking','pushing','pat on back','point finger','hugging','giving object','touch pocket',
           'shaking hands','walking towards','walking apart','put on headphone','take off headphone',
           'shoot at basket','bouce ball','tennis bat swing','juggle bat tennis ball','hush','flick hair',
           'thumb up','thumb down','make OK sign','make victory sign','staple book','couting money',
           'cutting nails','cutting paper','snap fingers','open bottle','sniff smell','squat down','toss a coin',
           'fold paper','bull up paper','play magic cube','apply cream on face','apply cream on hand','put on bag',
           'take off bag','put object into bag','take object out of bag','open a box','move heavy objects',
           'shake fist','throw up cap hat','capitulate','cross arms','arm circles','arm swings',
           'run on the spots','butt kicks','cross toe touch','side kick','yawn','stretch one self','blow nose',
           'hit with object','wield knife','knock over','grab stuff','shoot with gun','step on foot','high-five',
           'cheers and drink','carry object','take a photo','follow','whisper','exchange things',
           'support somebody','rock-paper-scissors']

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig("/content/drive/MyDrive/Colab Notebooks/vgg16/" + filename + '_plot.png')
	pyplot.close()

In [ ]:
x_train = None
y_train = None
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/vgg16/numpy/foot_to_foot_25/trainlabel_25_40_foot_to_foot.npy')
x_train = np.load('/content/drive/MyDrive/Colab Notebooks/vgg16/numpy/foot_to_foot_25/traintab_25_40_foot_to_foot.npy')
# for i in range(5):
#   rand_state = np.random.RandomState(i)
#   rand_state.shuffle(x_train)
#   rand_state.seed(i)
#   rand_state.shuffle(y_train)
# x_train = x_train[:4500000]
# x_train = np.reshape(x_train,(20,int(x_train.shape[0]/20),25,40,3))
# print(x_train.shape)

# y_train = y_train[:4500000]
# y_train = np.reshape(y_train,(20,int(y_train.shape[0]/20)))
# print(y_train.shape)

x_test = np.load('/content/drive/MyDrive/Colab Notebooks/vgg16/numpy/foot_to_foot_25/testtab_25_40_foot_to_foot.npy')[:500000]
y_test = np.load('/content/drive/MyDrive/Colab Notebooks/vgg16/numpy/foot_to_foot_25/testlabel_25_40_foot_to_foot.npy')[:500000]
# x_test = np.reshape(x_test,(20,int(x_test.shape[0]/20),25,40,3))
print(x_test.shape)
# y_test = np.reshape(y_test,(20,int(y_test.shape[0]/20)))
print(y_test.shape)

(500000, 25, 40, 3)
(500000,)


In [ ]:
print(y_train)

[77 77 77 ... 16 16 16]


In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.7.212.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.7.212.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
tf.keras.backend.clear_session()
#x_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).cache().shuffle(batch_size).repeat().batch(batch_size, drop_remainder=True)
#test_dataset = input_fn(x_train,y_train,batch_size)
with strategy.scope():
  vgg16_base = tf.keras.applications.VGG16(include_top=False,input_shape=(48,48,3))
  # freeze pretrained model
  vgg16_base.trainable = True
  print(vgg16_base.summary())
  model = tf.keras.models.Sequential(name='vgg16')
  model.add(Input(shape=(25,40,3)))
  model.add(Resizing(height=48,width=48))
  model.add(vgg16_base)
  model.add(Flatten())
  model.add(Dense(units=512,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(units=512,activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(units=120,activation="softmax"))
  model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=['accuracy'])
  model.summary()

# for j in range(100):
#   for i in range(x_train.shape[0]):
#     print("Epoque : " + str(j) + " Tab " + str(i))
#     model.fit(
#       x_train[i],
#       y_train[i],
#       epochs=1,
#       batch_size=batch_size,
#       steps_per_epoch=int(x_train[i].shape[0]/batch_size),
#       shuffle=True,
#       verbose=1
#     )


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [47]:
print(x_train.shape)
print(y_train.shape)
batch_size=360
model.fit(
      x_train,
      y_train,
      epochs=100,
      batch_size=batch_size,
      shuffle=True,
      verbose=1)
tf.keras.models.save_model(model,filepath='/content/drive/MyDrive/Colab Notebooks/vgg16/model_ntu_tpu/foot_to_foot/model_test.h5')

(492840, 25, 40, 3)
(492840,)
Epoch 1/100
1369/1369 [==============================] - 64s 34ms/step - loss: 4.2548 - accuracy: 0.0301
Epoch 2/100
1369/1369 [==============================] - 46s 34ms/step - loss: 3.4408 - accuracy: 0.0891
Epoch 3/100
1369/1369 [==============================] - 47s 34ms/step - loss: 2.8382 - accuracy: 0.2025
Epoch 4/100
1369/1369 [==============================] - 47s 34ms/step - loss: 2.2446 - accuracy: 0.3567
Epoch 5/100
1369/1369 [==============================] - 47s 34ms/step - loss: 1.7796 - accuracy: 0.4832
Epoch 6/100
1369/1369 [==============================] - 47s 34ms/step - loss: 1.4528 - accuracy: 0.5716
Epoch 7/100
1369/1369 [==============================] - 47s 34ms/step - loss: 1.2096 - accuracy: 0.6408
Epoch 8/100
1369/1369 [==============================] - 47s 34ms/step - loss: 1.0329 - accuracy: 0.6909
Epoch 9/100
1369/1369 [==============================] - 48s 35ms/step - loss: 0.8924 - accuracy: 0.7314
Epoch 10/100
1369/1369 [=

In [48]:
print(x_test.shape)

_, acc = model.evaluate(x_test,y_test,steps=50000,verbose=1)
print('> %.3f' % (acc * 100.0))

(500000, 25, 40, 3)
50000/50000 [==============================] - 795s 16ms/step - loss: 0.4845 - accuracy: 0.9128
> 91.277
